# Logical Agents

In [ ]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

 This notebook serves as a supporting material for the chapter **Logical Agents**. The notebooks illustrate the use of the code repository and demonstrate how the code can be extended to solve various related problems. We begin with overall agent design, introduce a simple new environment, the wumpus world, and illustrate the operation of knowledge-based agent. Then we have a look at the gernal principles of **logic** and the specifics of **propositional logic** and with well-developed inference technologies. At last, we combine the concept of knowledge-based agents with technology of propositional logic to build some simple agents for the wumpus world.

## Knowledge-Based Agents

The central component of a knowledge-based agent is its **knowledge base**, or **KB**. A knowledge base is a set of **sentences**. (Here “sentence” is used as a technical term. It is related but not identical to the sentences of English and other natural languages.) Each sentence is expressed in a language called a **knowledge representation language** and represents some assertion about the world. Sometimes we dignify a sentence with the name **axiom**, when the
sentence is taken as given without being derived from other sentences.

There must be a way to add new sentences to the knowledge base and a way to query what is known. The standard names for these operations are TELL and ASK, respectively.Both operations may involve **inference**—that is, deriving new sentences from old. Inference must obey the requirement that when one ASKs a question of the knowledge base, the answer should follow from what has been told (or TELLed) to the knowledge base previously.

The pseudocode below shows the outline of a knowledge-based agent program. Like all other agents, it takes a percept as input and returns an action. The agent maintaining knowlege base, KB, which may initially contain some **background knowledge*

In [2]:
from notebookUtils import *
pseudocode('KB-Agent')

### AIMA3e
__function__ KB-AGENT(_percept_) __returns__ an _action_  
&emsp;__persistent__: _KB_, a knowledge base  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_t_, a counter, initially 0, indicating time  

&emsp;TELL(_KB_, MAKE\-PERCEPT\-SENTENCE(_percept_, _t_))  
&emsp;_action_ &larr; ASK(_KB_, MAKE\-ACTION\-QUERY(_t_))  
&emsp;TELL(_KB_, MAKE\-ACTION\-SENTENCE(_action_, _t_))  
&emsp;_t_ &larr; _t_ + 1  
&emsp;__return__ _action_  

---
__Figure__ ?? A generic knowledge\-based agent. Given a percept, the agent adds the percept to its knowledge base, asks the knowledge base for the best action, and tells the knowledge base that it has in fact taken that action.

The implementation of the above pseudocode can be viewed [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/agent/KBAgent.java). This agent is implemented as an abstract agent which can be extended to construct other agents.

# Propositional Logic

We now present a simple but powerful logic called **propositional logic**. We cover the syntax of propositional logic and its semantics—the way in which the truth of sentences is determined. Then we look at **entailment**—the relation between a sentence and another sentence that follows from it—and see how this leads to a simple algorithm for logical inference. Everything takes place, of course, in the wumpus world.

### 1. Syntax

The syntax of propositional logic defines the allowable sentences. The atomic sentences consist of a single **proposition symbol**. Each such symbol stands for a proposition that can be true or false. We use symbols that start with an uppercase letter and may contain other letters or subscripts.The names are arbitrary but are often chosen to have some mnemonic value—we use W<sub>1, 3</sub> to stand for the proposition that the wumpus is in [1,3]. (Remember that symbols such as W<sub>1,3</sub> are *atomic*, i.e., W, 1,and 3 are not meaningful parts of the symbol.) There are two proposition symbols with fixed
meanings: *True* is the always-true proposition and *False* is the always-false proposition.

**Complex sentences** are constructed from simpler sentences, using parentheses and logical connectives. There are five connectives in common use:


* ¬ (not). A sentence such as ¬W<sub>1,3</sub> is called the **negation** of W<sub>1,3</sub>. A literal is either an atomic sentence (a **positive literal**) or a negated atomic sentence (a **negative literal**).

* ∧ (and). A sentence whose main connective is ∧, such as W<sub>1, 3</sub> ∧ P<sub>3, 1</sub>, is called a **conjunction**; its parts are the **conjuncts**.

* ∨ (or). A sentence using ∨, such as (W<sub>1, 3</sub> ∧ P<sub>3, 1</sub>)∨W<sub>2, 2</sub>, is a **disjunction** of the **disjuncts**
(W<sub>1, 3</sub> ∧ P<sub>3, 1</sub>) and W<sub>2, 2</sub>.

* ⇒ (implies). A sentence such as (W<sub>1, 3</sub> ∧ P<sub>3, 1</sub>) ⇒ ¬W<sub>2, 2</sub> is called an **implication** (or conditional). Its **premise** or **antecedent** is (W<sub>1, 3</sub> ∧ P<sub>3, 1</sub>), and its **conclusion** or **consequent** is ¬W<sub>2, 2</sub>. Implications are also known as **rules** or **if–then** statements. The implication symbol is sometimes written in other books as ⊃ or →.

* ⇔ (if and only if). The sentence W<sub>1, 3</sub> ⇔ ¬W<sub>2, 2</sub> is a biconditional. Some other books write this as ≡.

The implementation of forming **Complex Sentences** is [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/parsing/ast/ComplexSentence.java). The Complex sentence is implemented using [Sentence](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/parsing/ast/Sentence.java) and requires [Connective](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/parsing/ast/Connective.java) to form it properly.

### 2. Semantics

The semantics defines the rules for determining the truth of a sentence with respect to a particular model. In propositional logic, a model simply fixes the **truth value**—*true* or *false*—for every proposition symbol

The semantics for propositional logic must specify how to compute the truth value of any sentence, given a model. This is done recursively. All sentences are constructed from [AtomicSentence](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/parsing/ast/AtomicSentence.java) and the five connectives; therefore, we need to specify how to compute the truth of atomic sentences and how to compute the truth of sentences formed with each of the five connectives. Atomic sentences are easy:
* *True* is true in every model and *False* is false in every model.
* The truth value of every other proposition symbol must be specified directly in the model.

For complex sentences, we have five rules, which hold for any subsentences *P* and *Q* in any model *m* (here “iff” means “if and only if”):
* ¬*P* is true iff *P* is false in *m*.
* *P* ∧ *Q* is true iff both *P* and *Q* are true in *m*.
* *P* ∨ *Q* is true iff either *P* or *Q* is true in *m*.
* *P* ⇒ *Q* is true unless *P* is true and *Q* is false in *m*.
* *P* ⇔ *Q* is true iff *P* and *Q* are both true or both false in *m*.

## Truth Table Entailment

*Truth-Table Entails* performs a recursive enumeration of a finite space of assignments to symbols. The algorithm is **sound** because it implements directly the definition of entailment, and complete because it works for any KB
and α and always terminates—there are only finitely many models to examine.

Of course, “finitely many” is not always the same as “few.” If *KB* and *α* contain *n* symbols in all, then there are *2<sup>n<sup>* models. Thus, the time complexity of the algorithm is *O(2n)*. (The space complexity is only *O(n)* because the enumeration is depth-first.) Unfortunately, propositional entailment is co-NP-complete (i.e., probably no easier than NP-complete), so *every known inference algorithm for propositional logic has a worst-case
complexity that is exponential in the size of the input.*

In [1]:
from notebookUtils import *
pseudocode('TT-Entails')

### AIMA3e
__function__ TT-ENTAILS?(_KB_, _&alpha;_) __returns__ _true_ or _false_  
&emsp;__inputs__: _KB_, the knowledge base, a sentence in propositional logic  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_&alpha;_, the query, a sentence in propositional logic  

&emsp;_symbols_ &larr; a list of the propositional symbols in _KB_ and _&alpha;_  
&emsp;__return__ TT-CHECK-ALL(_KB_, _&alpha;_, _symbols_, { })  

---
__function__ TT-CHECK-ALL(_KB_, _&alpha;_, _symbols_, _model_) __returns__ _true_ or _false_  
&emsp;__if__ EMPTY?(_symbols_) __then__  
&emsp;&emsp;&emsp;__if__ PL-TRUE?(_KB_, _model_) __then return__ PL-TRUE?(_&alpha;_, _model_)  
&emsp;&emsp;&emsp;__else return__ _true_&emsp;_//_&emsp;_when KB is false, always return true_  
&emsp;__else do__  
&emsp;&emsp;&emsp;_P_ &larr; FIRST(_symbols_)  
&emsp;&emsp;&emsp;_rest_ &larr; REST(_symbols_)  
&emsp;&emsp;&emsp;__return__(TT-CHECK-ALL(_KB_, _&alpha;_, _rest_, _model_ &cup; { _P_ = _true_ })  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;__and__  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;TT-CHECK-ALL(_KB_, _&alpha;_, _rest_, _model_ &cup; { _P_ = _false_ }))  
   

---
__Figure__ ?? A truth-table enumeration algorithm for deciding propositional entailment. (TT stands for truth table.) PL-TRUE? returns _true_ if a sentence holds within a model. The variable _model_ represents a partial model \- an assignment to some of the symbols. The keyword "__and__" is used here as a logical operation on its two arguments, returning _true_ or _false_.